### Nettoyage et Prétraitement des Données - `2_preprocessing.ipynb`

#### Objectif :
Préparer les données en nettoyant le texte pour éliminer les éléments superflus, effectuer une tokenisation, et transformer le texte en une forme exploitable pour les modèles de machine learning.

In [23]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Téléchargement des ressources nécessaires de NLTK
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt') 


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
# Charger le DataFrame à partir du fichier pickle
df = pd.read_pickle('df_data.pkl')

# Afficher les premières lignes pour vérifier que les données sont chargées
df.head()

,tweet,label,tweet_length
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,115
1,is upset that he can't update his Facebook by ...,1,111
2,@Kenichan I dived many times for the ball. Man...,1,89
3,my whole body feels itchy and like its on fire,0,47
4,"@nationwideclass no, it's not behaving at all....",1,111


#### Fonction de nettoyage du texte

- **Supprimer les URL, mentions et hashtags** :

Les tweets peuvent contenir des éléments comme des URL, des mentions (`@username`), et des hashtags (`#hashtag`) qui ne sont pas utiles pour le modèle de machine learning. Ces éléments sont supprimés pour rendre le texte plus propre.

- **Suppression de la ponctuation et des chiffres** : 
Cela permet de simplifier le texte en retirant les caractères inutiles pour l'analyse.

In [26]:
# Fonction pour nettoyer un texte
def nettoyer_texte(texte):
    # Suppression des URL
    texte = re.sub(r'http\S+', '', texte)  # Supprimer les URL

    # Suppression des mentions (@username)
    texte = re.sub(r'@\w+', '', texte)  # Supprimer les mentions

    # Suppression des hashtags (#hashtag)
    texte = re.sub(r'#\w+', '', texte)  # Supprimer les hashtags

    # Suppression de la ponctuation
    texte = texte.translate(str.maketrans('', '', string.punctuation))

    # Suppression des chiffres
    texte = re.sub(r'\d+', '', texte)  # Supprimer les chiffres

    # Conversion en minuscules
    texte = texte.lower()

    return texte

# Appliquer le nettoyage à la colonne 'tweet'
df['cleaned_tweet'] = df['tweet'].apply(nettoyer_texte)

# Afficher les premières lignes pour vérifier que les données sont chargées
df.head()


,tweet,label,tweet_length,cleaned_tweet
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1,115,awww thats a bummer you shoulda got david ...
1,is upset that he can't update his Facebook by ...,1,111,is upset that he cant update his facebook by t...
2,@Kenichan I dived many times for the ball. Man...,1,89,i dived many times for the ball managed to sa...
3,my whole body feels itchy and like its on fire,0,47,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all....",1,111,no its not behaving at all im mad why am i he...


In [27]:
# Vectorisation des tweets nettoyés en utilisant TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Appliquer TF-IDF sur les tweets nettoyés
X = tfidf_vectorizer.fit_transform(df['cleaned_tweet'])  # Assurez-vous que 'cleaned_tweet' est la bonne colonne
y = df['label']  # Label de la classification

#### Tokenisation et suppression des mots vides (stopwords)
La tokenisation consiste à découper le texte en mots (ou tokens). Cela permet de transformer le texte brut en une séquence de mots exploitables par les modèles.

In [28]:
# Liste des stop words
stop_words = set(stopwords.words('english'))

# Fonction de tokenisation et suppression des stop words
def tokeniser_et_supprimer_stopwords(texte):
    # Tokenisation
    tokens = word_tokenize(texte)
    
    # Suppression des stop words
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]
    
    return tokens

# Appliquer la tokenisation et la suppression des stop words
df['tokens'] = df['cleaned_tweet'].apply(tokeniser_et_supprimer_stopwords)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\benew/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\benew\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/nltk_data'
**********************************************************************


#### 2.5. Lemmatisation (ou Stemming)

La lemmatisation est un processus qui permet de réduire les mots à leur forme de base (par exemple, "manger" devient "mange"). Cela permet de traiter différentes formes d'un même mot comme une seule entité.

In [30]:
# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction de lemmatisation
def lemmatiser(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# Appliquer la lemmatisation
df['lemmatised_tokens'] = df['tokens'].apply(lemmatiser)


KeyError: 'tokens'

#### 2.6. Transformation en forme numérique avec TF-IDF

Une fois le texte nettoyé et transformé en tokens lemmatisés, vous pouvez utiliser un `TfidfVectorizer` pour transformer ces tokens en une représentation numérique que les modèles de machine learning peuvent comprendre.

In [17]:
# Transformation du texte en une matrice de caractéristiques (TF-IDF)
vectorizer = TfidfVectorizer(max_features=1000)  # Limiter à 1000 caractéristiques

# Appliquer la transformation
X = vectorizer.fit_transform(df['cleaned_tweet'])

# Convertir la matrice sparse en DataFrame pour plus de lisibilité
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Ajouter la colonne 'label' au DataFrame pour l'analyse
X_df['label'] = df['label']


In [18]:
# Affichage des premières lignes de données traitées
X_df.head()

,able,about,absolutely,account,actually,add,after,afternoon,again,ago,...,youll,your,youre,yours,yourself,youtube,youve,yummy,yup,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


### 2.7. Affichage du résultat






   
### Conclusion

Ce prétraitement permet de nettoyer les données textuelles et de les préparer pour les modèles de machine learning. Vous pouvez maintenant utiliser les données nettoyées et transformées pour entraîner des modèles de classification ou d'autres tâches de traitement de texte.

In [31]:
# Sauvegarder df dans un fichier pickle
df.to_pickle('df_data_cleaned.pkl')
